# 套路

别误会，并非取自“套路太深”，而是习武之人都懂的武术的“套路”和“散打”两大类的“套路”，表示“一整套规定的动作”，如华山基本剑法：白云出岫、有凤来仪、天绅倒悬、白虹贯日、苍松迎客、金雁横空、无边落木、青山隐隐、古柏森森、无双无对、金玉满堂。

操作TensorFlow也必须明白一些术语和规程：

- 图（Graph）：表示计算任务
- 会话（Session）：表示执行图的上下文
- 张量（Tensor）：表示数据
- 变量（Variable）：维护状态
- Feed/Fetch：为任意操作赋值或获取其中的数据

## 综述

总之，TF是一套变成系统，用图来表示计算任务，图中的节点成为OP（OPeration）。一个OP有**0**个或多个张量，执行计算产出**0**个或多个张量。每个张量是一种类型化的多维数组。

举个例子，某组图像集表示为一个四维浮点数组： $$[batch, height, width, channels]$$ 。

而TF描绘了计算过程。

为了进行计算，图必须在会话中被启动。

会话把图的OP分发到CPU/GPU等设备上，同时提供执行OP的方法。

这些方法执行完成，将产生新的张量并返回。

- Python中返回的是numpy的ndarray对象
- C/C++中返回的是tensorflow::Tensor实例

### 计算图

通常分阶段组织程序。

在构建阶段，OP的执行步骤被描述成一个图。

在执行阶段，在会话中执行图中的OP。

举个例子，一般在构建阶段创建一个图来表示和训练神经网络，接着在执行阶段反复执行图中的训练OP。

TF目前支持C/C++/Python/JS等编程语言，而Python是最早被支持且支持德最全面。

*会话库是一致的*。

### 构建图

- 第一步：创建源OP

    源OP不需要任何输入，如常量（Constant）。
    
    源OP的输出被传递给别的OP做计算。
    
    Python库中，OP构造器的返回至代表构造出的OP的输出，可以传递给别的OP的构造器作为输入。
    
    Python库中，有个默认图，OP构造器可为其添加节点，该默认图可满足大多程序，参考Graph类了解[TF如何管理多个图](./TF如何管理多个图.ipynb)。

In [1]:
import tensorflow as tf

# 创建一个常量OP、产出一个1*2的矩阵，该OP作为一个节点
# 把该OP添加到默认图中

# 构造器的返回值代表该常量OP的返回值

matrix_1_2 = tf.constant([[3., 3.]]) # 一行两列

matrix_2_1 = tf.constant([[2.], [2.]]) # 两行一列

# 创建矩阵乘积OP，将matrix_1_2和matrix_2_1作为输入、返回值就是矩阵乘积的结果

product = tf.matmul(matrix_1_2, matrix_2_1)

此时的默认图中有三个节点：

- constant OP: 2
- matmul OP: 1

为了真正地进行矩阵乘积计算并得到结果，就必须在会话中启动这个图。

### 在一个会话中启动图

构造阶段完成后方可启动图。

- 第一步：创建一个Session对象

    如无任何创建参数，会话构造器将启动默认图。
    
    参考Session类了解完整的会话API。

In [2]:
# 启动默认图

sess = tf.Session()

# 调用sess的run()方法来执行矩阵乘积OP，传入product作为该方法的参数
# 这里提到的product代表矩阵乘积OP的输出，传入它是想向方法表明：希望取回矩阵乘积OP的输出

# 整个执行过程是自动的，会话负责传递OP所需的全部输入
# OP的执行通常是并发的

# 函数调用run(product)触发图中的三个OP的执行

# 返回值result是个numpy的ndarray对象

result = sess.run(product)
print(result)

# 任务完成关闭会话

sess.close()

[[ 12.]]


关闭Session对象以释放资源，除了显式调用close之外，也可以用with代码块。

In [3]:
with tf.Session() as sess:
    result = sess.run([product])
    print(result)

[array([[ 12.]], dtype=float32)]


在实现上，TF将图定义转换成分布式执行的OP簇，以充分利用可用的计算资源（CPU/GPU）。

一般不需要显式指定CPU或GPU，TF能自动检测。

若检测到GPU，TF会尽可能利用找到的第一块GPU来执行OP。

若机器上有多个GPU，除了第一块之外，别的GPU默认不参与计算。

为了让TF能利用这些GPU，必须将OP明确指派执行设备，通过with...Device实现。

In [4]:
DN = 0

with tf.Session() as sess:
    # with tf.device("/gpu:%d" % DN): # TODO: 为何本地有个NVIDIA-GeForce-GT-705的显卡却报错找不到第一块显卡
    with tf.device("/cpu:%d" % DN):
        matrix_1_2 = tf.constant([[3., 3.]])
        matrix_2_1 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix_1_2, matrix_2_1)
        sess.run(product)
        print(product)

Tensor("MatMul_1:0", shape=(1, 1), dtype=float32, device=/device:CPU:0)


设备采用串码标识，目前支持的设备：

- /cpu:0 标识机器上第一块CPU
- /gpu:0 标识机器上第一块GPU

*以此类推*。

参考[TF使用GPU](./TF使用GPU.ipynb)了解tensorflow-gpu的更多详情。

### 交互式

文档中的Python示例使用一个会话Session来启动图（Graph），并调用Session.run()方法来执行每个OP。

为了便于应用如IPython之类的Python交互式环境，可以：

- 使用InteractiveSession代替Session类
- 使用Tensor.eval()和Operation.run()代替Session.run

能避免通过一个变量来持有会话。

In [6]:
# 进入一个交互式TF会话

sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0]) # 变量、一行两列
a = tf.constant([3.0, 3.0]) # 常量、一行两列

# 采用初始化器（initializer）OP的run()方法初始化x

x.initializer.run()

# 添加一个减法（sub）OP、从x减去a、运行该OP，输出结果

sub = tf.subtract(x, a)
print(sub.eval())

[-2. -1.]


### 张量（Tensor）

这是TF中的统一数据结构，操作之间传递的都是张量。

可以看作是N维数组。

一个张量包含：

- 阶rank
- 数据类型type
- 形状shape

详见[张量的阶、数据类型和形状](./TF中张量的阶、数据类型和形状.ipynb)。

### 变量

详见Variable类，参考[TF的变量](./TF的变量.ipynb)。

变量来维护图执行过程中的状态。

看一个的计数器的例程：

In [8]:
# 创建一个变量，初始化为零（标量）

state = tf.Variable(0, name="counter")

# 创建一个让state加一的OP

one = tf.constant(1)

egg = tf.add(state, one)

update = tf.assign(state, egg)

# 启动默认图之前，变量必须初始化（通过初始化OP实现）

# 添加初始化OP到图中

iOP = tf.global_variables_initializer()

# 启动图，运行所有OP

with tf.Session() as sess:
    sess.run(iOP)
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3
